In [25]:
import pandas

# Достаем таблицу
table = pandas.read_csv('cian_parsed.csv')
table = table.drop_duplicates()
# Убираем ненужные нам данные
table.drop(['deal_type', 'url', 'author', 'accommodation_type', 'house_number', 'phone', 'heating_type'], axis=1, inplace=True)
# Отображаем
table.head(10000)

,author_type,location,floor,floors_count,rooms_count,total_meters,price,year_of_construction,object_type,house_material_type,finish_type,living_meters,kitchen_meters,district,street,underground,residential_complex
0,developer,Москва,2,16,1,45.00,23274675,2024,-1,Монолитно-кирпичный,Без отделки,"16,4 м²","14,6 м²",Филевский парк,Береговой проезд,Фили,Береговой-2
1,real_estate_agent,Москва,9,40,1,41.50,32000000,-1,-1,-1,-1,"23,4 м²","4,9 м²",Хорошевский,Ленинградский проспект,ЦСКА,Прайм Парк
2,real_estate_agent,Москва,11,12,1,33.50,8000000,1972,-1,-1,-1,17 м²,7 м²,Чертаново Южное,3-й Дорожный проезд,Улица Академика Янгеля,NaN
3,developer,Москва,3,26,1,23.90,22602971,2027,-1,Монолитный,"Без отделки, предчистовая, чистовая",-1,-1,Даниловский,NaN,Павелецкая,Хай Лайф ЖК
4,real_estate_agent,Москва,13,41,1,41.50,31000000,-1,-1,-1,-1,"23,4 м²","4,9 м²",Хорошевский,Ленинградский проспект,Аэропорт,Прайм Парк
5,real_estate_agent,Москва,9,14,1,38.50,7000000,1993,-1,-1,-1,18 м²,8 м²,Крюково,Зеленоград,Зеленоград — Крюково,NaN
6,developer,Москва,7,36,1,40.90,30348986,2027,-1,-1,Неизвестно,-1,-1,Беговой,Ленинградский проспект,Белорусская,Slava
7,real_estate_agent,Москва,3,20,1,59.00,61800000,2023,-1,-1,-1,39 м²,15 м²,Пресненский,Костикова,Улица 1905 года,Lucky
8,real_estate_agent,Москва,2,2,1,34.50,6400000,-1,-1,-1,-1,"19,5 м²",10 м²,Марьино,Перерва,Перерва,NaN
9,developer,Москва,2,27,1,45.70,21798900,2026,-1,-1,Неизвестно,"26,8 м²","7,8 м²",Басманный,Фридриха Энгельса,Бауманская,Дом 56
